In [51]:
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine, text
import sqlite3

In [52]:
jpy_csv = Path("Resources/JPY_USD_Prices.csv")
jpy_df = pd.read_csv(jpy_csv)
jpy_df.head()

,Date,Open,High,Low,Close
0,01/01/25,157.21,157.44,157.19,157.40
1,12/31/24,156.84,157.55,156.03,157.21
2,12/30/24,157.87,158.08,156.67,156.84
3,12/27/24,158.00,158.00,157.35,157.84
4,12/26/24,157.19,158.09,157.08,158.00


In [53]:
euro_csv = Path("Resources/EURO_USD_Prices.csv")
euro_df = pd.read_csv(jpy_csv)
euro_df.head()

,Date,Open,High,Low,Close
0,01/01/25,157.21,157.44,157.19,157.40
1,12/31/24,156.84,157.55,156.03,157.21
2,12/30/24,157.87,158.08,156.67,156.84
3,12/27/24,158.00,158.00,157.35,157.84
4,12/26/24,157.19,158.09,157.08,158.00


In [54]:
cad_csv = Path("Resources/CAD_USD_Prices.csv")
cad_df = pd.read_csv(cad_csv)
cad_df.head()

,Date,Open,High,Low,Close
0,01/01/25,1.4387,1.4407,1.4381,1.4392
1,12/31/24,1.4359,1.4412,1.4339,1.4385
2,12/30/24,1.4421,1.4432,1.4345,1.4357
3,12/27/24,1.4407,1.4450,1.4376,1.4413
4,12/26/24,1.4379,1.4423,1.4349,1.4408


In [60]:
foreign_merge_df = pd.merge(jpy_df, euro_df, on='Date')
foreign_merge_df.head()

,Date,Open_x,High_x,Low_x,Close_x,Open_y,High_y,Low_y,Close_y
0,01/01/25,157.21,157.44,157.19,157.40,157.21,157.44,157.19,157.40
1,12/31/24,156.84,157.55,156.03,157.21,156.84,157.55,156.03,157.21
2,12/30/24,157.87,158.08,156.67,156.84,157.87,158.08,156.67,156.84
3,12/27/24,158.00,158.00,157.35,157.84,158.00,158.00,157.35,157.84
4,12/26/24,157.19,158.09,157.08,158.00,157.19,158.09,157.08,158.00


In [55]:
foreign_merge_df = pd.merge(jpy_df, euro_df, on='Date')
foreign_df = pd.merge(foreign_merge_df, cad_df, on="Date")
foreign_df = foreign_merge_df.sort_index(ascending=True) #date in ascending order
foreign_df

,Date,Open_x,High_x,Low_x,Close_x,Open_y,High_y,Low_y,Close_y
0,01/01/25,157.21,157.44,157.19,157.40,157.21,157.44,157.19,157.40
1,12/31/24,156.84,157.55,156.03,157.21,156.84,157.55,156.03,157.21
2,12/30/24,157.87,158.08,156.67,156.84,157.87,158.08,156.67,156.84
3,12/27/24,158.00,158.00,157.35,157.84,158.00,158.00,157.35,157.84
4,12/26/24,157.19,158.09,157.08,158.00,157.19,158.09,157.08,158.00
...,...,...,...,...,...,...,...,...,...
258,01/05/24,144.63,145.96,143.81,144.62,144.63,145.96,143.81,144.62
259,01/04/24,143.29,144.85,142.87,144.63,143.29,144.85,142.87,144.63
260,01/03/24,141.99,143.73,141.86,143.28,141.99,143.73,141.86,143.28
261,01/02/24,140.88,142.22,140.82,141.99,140.88,142.22,140.82,141.99


In [ ]:
#renaming the columns
# foreign_df.rename(columns={'High_x': 'Euro_High', 'High_x': 'Euro_High', 'Low_'}, inplace=True)

In [56]:
# Save to CSV
foreign_df.to_csv("Resources/Foreign_Currency.csv", index=False)  # index=False removes the extra index column

print("CSV file saved successfully!")

CSV file saved successfully!


In [61]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("currency.db")  # Creates 'currency.db' file
cursor = conn.cursor()

# Save DataFrame to SQLite table (if table doesn't exist, it will be created)
foreign_df.to_sql("foreign_currency", conn, if_exists="replace", index=False)  # 'replace' overwrites if exists

# Query the database to check
query_result = pd.read_sql("SELECT * FROM foreign_currency;", conn)
print(query_result)  # Display first 5 rows

# Close the connection
conn.close()
print("Database created successfully!")

         Date   Open_x   High_x   Low_x   Close_x   Open_y   High_y   Low_y  \
0    01/01/25   157.21   157.44  157.19    157.40   157.21   157.44  157.19   
1    12/31/24   156.84   157.55  156.03    157.21   156.84   157.55  156.03   
2    12/30/24   157.87   158.08  156.67    156.84   157.87   158.08  156.67   
3    12/27/24   158.00   158.00  157.35    157.84   158.00   158.00  157.35   
4    12/26/24   157.19   158.09  157.08    158.00   157.19   158.09  157.08   
..        ...      ...      ...     ...       ...      ...      ...     ...   
258  01/05/24   144.63   145.96  143.81    144.62   144.63   145.96  143.81   
259  01/04/24   143.29   144.85  142.87    144.63   143.29   144.85  142.87   
260  01/03/24   141.99   143.73  141.86    143.28   141.99   143.73  141.86   
261  01/02/24   140.88   142.22  140.82    141.99   140.88   142.22  140.82   
262  01/01/24   141.03   141.41  140.82    140.88   141.03   141.41  140.82   

      Close_y  
0      157.40  
1      157.21  
2  

In [58]:
# jpy_db = Path("Resources/JPY_USD_Prices.sqlite") #refernce module 10 activities
# conn = create_engine(f"sqlite:///{jpy_db}").connect()
# conn.close()

For the D3 website portion

In [ ]:
# will need to convert the sqlite file into a json to import and use for d3
import sqlite3
import json

# Connect to SQLite database
conn = sqlite3.connect("currency.db")
cursor = conn.cursor()

# Query data from SQLite
cursor.execute("SELECT * FROM data_table;")  # Replace with your table name
rows = cursor.fetchall()

# Get column names
columns = [desc[0] for desc in cursor.description]

# Convert to list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Save as JSON
with open("data.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

conn.close()
print("JSON file created successfully!")
